<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Word2vect


In [ ]:
import numpy as np

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [ ]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [ ]:
corpus_terminos = []
for doc in corpus:
  terminos = doc.split(" ")
  corpus_terminos.append(terminos)

corpus_terminos = np.array(corpus_terminos)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [ ]:
vocab_completo_list = np.sum(corpus_terminos)
diccionario = {}
for palabra in vocab_completo_list:
  if palabra in diccionario:
    diccionario[palabra] += 1
  else:
    diccionario[palabra] = 1

for palabra in diccionario:
  frecuencia = diccionario[palabra]
  print(f"La palabra '{palabra}' tiene una frecuencia de {frecuencia}")

palabras = np.unique(vocab_completo_list)

La palabra 'que' tiene una frecuencia de 1
La palabra 'dia' tiene una frecuencia de 2
La palabra 'es' tiene una frecuencia de 2
La palabra 'hoy' tiene una frecuencia de 2
La palabra 'martes' tiene una frecuencia de 3
La palabra 'el' tiene una frecuencia de 1
La palabra 'de' tiene una frecuencia de 1
La palabra 'muchas' tiene una frecuencia de 1
La palabra 'gracias' tiene una frecuencia de 1


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [ ]:
def OHE (corpus_term, vocab):
  matriz = np.zeros((len(corpus_term), len(vocab)))
  for i, d in enumerate(corpus_term):
    for j, t in enumerate(palabras):
      matriz[i, j] = t in d

  return matriz
OHot = OHE(corpus_terminos, palabras)
print(OHot)


[[0. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 0.]]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [ ]:
def vect_frec(corpus_term, vocab):
  matriz = np.zeros((len(corpus_term), len(vocab)))
  for i, d in enumerate(corpus_term):
    for j, t in enumerate(palabras):
      matriz[i, j] = d.count(t)
  
  return matriz
VFrec = vect_frec(corpus_terminos, palabras)
print(VFrec)

[[0. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 1. 0. 1. 2. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 1. 0.]]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [ ]:
N = len(corpus_terminos)
DF = OHot.sum(axis=0)
IDF = np.log2(N/DF)
TF_IDF = (VFrec * IDF).round(decimals=2)
print(TF_IDF)


[[0.   0.58 0.   0.58 0.   0.58 0.   0.   1.58]
 [1.58 0.58 1.58 0.58 0.   0.58 1.17 0.   0.  ]
 [0.   0.   0.   0.   1.58 0.   0.58 1.58 0.  ]]


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [ ]:
comp = np.zeros((N, N))

for i in range(N):
  for j in range(N):
    comp[i,j] = cosine_similarity(TF_IDF[i], TF_IDF[j])

print(comp)

[[1.         0.19853442 0.        ]
 [0.19853442 1.         0.1082736 ]
 [0.         0.1082736  1.        ]]


In [ ]:
comp_up = np.triu(comp + 1e-15, k=1)
print(comp_up.round(decimals=4))

[[0.     0.1985 0.    ]
 [0.     0.     0.1083]
 [0.     0.     0.    ]]


In [ ]:
lim = int(N*(N-1)/2)
order = np.array(np.unravel_index(np.argsort(comp_up.flatten() * -1), comp_up.shape))[:, :lim]
print(order)

[[0 1 0]
 [1 2 2]]


In [ ]:
for i in range(lim):
  print("Cosine similarity: " + str(comp[order[0, i]][order[1, i]].round(decimals=4)) + " | " + corpus[order[0, i]] + " <-> " + corpus[order[1, i]])

Cosine similarity: 0.1985 | que dia es hoy <-> martes el dia de hoy es martes
Cosine similarity: 0.1083 | martes el dia de hoy es martes <-> martes muchas gracias
Cosine similarity: 0.0 | que dia es hoy <-> martes muchas gracias
